In [1]:
import findspark
import configparser
from pyspark.sql import SparkSession,functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import *
from pyspark.sql.functions import col, ltrim, rtrim, trim

In [2]:
spark = (SparkSession.builder 
                     .appName("homework4_4")
                     .master("local[2]")
                     .getOrCreate()) 

2023-04-07 16:35:10,073 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_credits = (spark.read
                   .format("csv")
                   .option("header", True)
                   .option("inferSchema", True)
                   .load("file:////home/train/datasets/tmdb_5000_movies_and_credits"))

In [5]:
df_credits.limit(5).toPandas()

,movie_id,title,cast,crew
0,19995,Avatar,"""[{""""cast_id"""": 242","""""character"""": """"Jake Sully"""""
1,285,Pirates of the Caribbean: At World's End,"""[{""""cast_id"""": 4","""""character"""": """"Captain Jack Sparrow"""""
2,206647,Spectre,"""[{""""cast_id"""": 1","""""character"""": """"James Bond"""""
3,49026,The Dark Knight Rises,"""[{""""cast_id"""": 2","""""character"""": """"Bruce Wayne / Batman"""""
4,49529,John Carter,"""[{""""cast_id"""": 5","""""character"""": """"John Carter"""""


In [6]:
df_credits2 = (spark.read
                    .format("csv")
                    .option("header", True)
                    .option("sep", ",")
                    .option("inferSchema", True)
                    .option("escape", "\"")
                    .load("file:////home/train/datasets/tmdb_5000_movies_and_credits/tmdb_5000_credits.csv"))

In [70]:
df_credits2.limit(20).toPandas()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,"[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,"[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,"[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


2023-04-07 04:39:37,440 WARN spark.HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2278760 ms exceeds timeout 120000 ms
2023-04-07 04:39:37,565 WARN spark.SparkContext: Killing executors is not supported by current scheduler.


In [29]:
df_credits_2 = df_credits.withColumn("cast", regexp_extract(df_credits['cast'], r'([0-9]+)', 1))

In [30]:
df_credits_2.show()

+--------+--------------------+----+--------------------+
|movie_id|               title|cast|                crew|
+--------+--------------------+----+--------------------+
|   19995|              Avatar| 242| ""character"": "...|
|     285|Pirates of the Ca...|   4| ""character"": "...|
|  206647|             Spectre|   1| ""character"": "...|
|   49026|The Dark Knight R...|   2| ""character"": "...|
|   49529|         John Carter|   5| ""character"": "...|
|     559|        Spider-Man 3|  30| ""character"": "...|
|   38757|             Tangled|  34| ""character"": "...|
|   99861|Avengers: Age of ...|  76| ""character"": "...|
|     767|Harry Potter and ...|   3| ""character"": "...|
|  209112|Batman v Superman...|  18| ""character"": "...|
|    1452|    Superman Returns|   3| ""character"": "...|
|   10764|   Quantum of Solace|   1| ""character"": "...|
|      58|Pirates of the Ca...|  37| ""character"": "...|
|   57201|     The Lone Ranger|   4| ""character"": "...|
|   49521|    

In [32]:
df_credits_3 = df_credits_2.withColumn("crew", F.regexp_replace("crew", '""character"": "', ""))
df_credits_3.show()

+--------+--------------------+----+--------------------+
|movie_id|               title|cast|                crew|
+--------+--------------------+----+--------------------+
|   19995|              Avatar| 242|       "Jake Sully""|
|     285|Pirates of the Ca...|   4| "Captain Jack Sp...|
|  206647|             Spectre|   1|       "James Bond""|
|   49026|The Dark Knight R...|   2| "Bruce Wayne / B...|
|   49529|         John Carter|   5|      "John Carter""|
|     559|        Spider-Man 3|  30| "Peter Parker / ...|
|   38757|             Tangled|  34| "Flynn Rider (vo...|
|   99861|Avengers: Age of ...|  76| "Tony Stark / Ir...|
|     767|Harry Potter and ...|   3|     "Harry Potter""|
|  209112|Batman v Superman...|  18| "Bruce Wayne / B...|
|    1452|    Superman Returns|   3| "Superman / Clar...|
|   10764|   Quantum of Solace|   1|       "James Bond""|
|      58|Pirates of the Ca...|  37| "Captain Jack Sp...|
|   57201|     The Lone Ranger|   4|            "Tonto""|
|   49521|    

In [55]:
df_credits_4= df_credits_3.withColumn("crew", F.translate("crew", '"', " "))
df_credits_4.show()  

+--------+--------------------+----+--------------------+
|movie_id|               title|cast|                crew|
+--------+--------------------+----+--------------------+
|   19995|              Avatar| 242|        Jake Sully  |
|     285|Pirates of the Ca...|   4|  Captain Jack Sp...|
|  206647|             Spectre|   1|        James Bond  |
|   49026|The Dark Knight R...|   2|  Bruce Wayne / B...|
|   49529|         John Carter|   5|       John Carter  |
|     559|        Spider-Man 3|  30|  Peter Parker / ...|
|   38757|             Tangled|  34|  Flynn Rider (vo...|
|   99861|Avengers: Age of ...|  76|  Tony Stark / Ir...|
|     767|Harry Potter and ...|   3|      Harry Potter  |
|  209112|Batman v Superman...|  18|  Bruce Wayne / B...|
|    1452|    Superman Returns|   3|  Superman / Clar...|
|   10764|   Quantum of Solace|   1|        James Bond  |
|      58|Pirates of the Ca...|  37|  Captain Jack Sp...|
|   57201|     The Lone Ranger|   4|             Tonto  |
|   49521|    

In [61]:
df_credits_5 = df_credits_4.withColumn("k", ltrim(col("movie_id")))

df_credits_5.show()  

+--------+--------------------+----+--------------------+------+
|movie_id|               title|cast|                crew|     k|
+--------+--------------------+----+--------------------+------+
|   19995|              Avatar| 242|        Jake Sully  | 19995|
|     285|Pirates of the Ca...|   4|  Captain Jack Sp...|   285|
|  206647|             Spectre|   1|        James Bond  |206647|
|   49026|The Dark Knight R...|   2|  Bruce Wayne / B...| 49026|
|   49529|         John Carter|   5|       John Carter  | 49529|
|     559|        Spider-Man 3|  30|  Peter Parker / ...|   559|
|   38757|             Tangled|  34|  Flynn Rider (vo...| 38757|
|   99861|Avengers: Age of ...|  76|  Tony Stark / Ir...| 99861|
|     767|Harry Potter and ...|   3|      Harry Potter  |   767|
|  209112|Batman v Superman...|  18|  Bruce Wayne / B...|209112|
|    1452|    Superman Returns|   3|  Superman / Clar...|  1452|
|   10764|   Quantum of Solace|   1|        James Bond  | 10764|
|      58|Pirates of the 

In [62]:
df_cast = (spark.read
                .format("csv")
                .option("header", True)
                .option("inferSchema", True)
                .load("file:////home/train/datasets/tmdb_5000_movies_and_credits/tmdb_5000_movies.csv"))

In [63]:
df_cast.limit(5).toPandas()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"""[{""""id"""": 28","""""name"""": """"Action""""}","{""""id"""": 12","""""name"""": """"Adventure""""}","{""""id"""": 14","""""name"""": """"Fantasy""""}","{""""id"""": 878","""""name"""": """"Science Fiction""""}]""",http://www.avatarmovie.com/,19995,"""[{""""id"""": 1463","""""name"""": """"culture clash""""}","{""""id"""": 2964","""""name"""": """"future""""}","{""""id"""": 3386","""""name"""": """"space war""""}","{""""id"""": 3388","""""name"""": """"space colony""""}","{""""id"""": 3679"
1,300000000,"""[{""""id"""": 12","""""name"""": """"Adventure""""}","{""""id"""": 14","""""name"""": """"Fantasy""""}","{""""id"""": 28","""""name"""": """"Action""""}]""",http://disney.go.com/disneypictures/pirates/,285,"""[{""""id"""": 270","""""name"""": """"ocean""""}","{""""id"""": 726","""""name"""": """"drug abuse""""}","{""""id"""": 911","""""name"""": """"exotic island""""}","{""""id"""": 1319","""""name"""": """"east india trading company""""}","{""""id"""": 2038","""""name"""": """"love of one's life""""}","{""""id"""": 2052"
2,245000000,"""[{""""id"""": 28","""""name"""": """"Action""""}","{""""id"""": 12","""""name"""": """"Adventure""""}","{""""id"""": 80","""""name"""": """"Crime""""}]""",http://www.sonypictures.com/movies/spectre/,206647,"""[{""""id"""": 470","""""name"""": """"spy""""}","{""""id"""": 818","""""name"""": """"based on novel""""}","{""""id"""": 4289","""""name"""": """"secret agent""""}","{""""id"""": 9663","""""name"""": """"sequel""""}","{""""id"""": 14555","""""name"""": """"mi6""""}","{""""id"""": 156095"
3,250000000,"""[{""""id"""": 28","""""name"""": """"Action""""}","{""""id"""": 80","""""name"""": """"Crime""""}","{""""id"""": 18","""""name"""": """"Drama""""}","{""""id"""": 53","""""name"""": """"Thriller""""}]""",http://www.thedarkknightrises.com/,49026,"""[{""""id"""": 849","""""name"""": """"dc comics""""}","{""""id"""": 853","""""name"""": """"crime fighter""""}","{""""id"""": 949","""""name"""": """"terrorist""""}","{""""id"""": 1308","""""name"""": """"secret identity""""}","{""""id"""": 1437"
4,260000000,"""[{""""id"""": 28","""""name"""": """"Action""""}","{""""id"""": 12","""""name"""": """"Adventure""""}","{""""id"""": 878","""""name"""": """"Science Fiction""""}]""",http://movies.disney.com/john-carter,49529,"""[{""""id"""": 818","""""name"""": """"based on novel""""}","{""""id"""": 839","""""name"""": """"mars""""}","{""""id"""": 1456","""""name"""": """"medallion""""}","{""""id"""": 3801","""""name"""": """"space travel""""}","{""""id"""": 7376","""""name"""": """"princess""""}","{""""id"""": 9951"
